# Reference states

In this lesson, we will explore how we can initialize our system with a [reference state][reference] to help our variational algorithm converge faster. First, we will learn how to construct a reference state manually, and then explore several standard options that can be used in a variational algorithm.

![Diagram of options for references states including default, application-specific, and quantum.](/learning/images/courses/variational-algorithm-design/reference-states/reference-workflow.svg)

[reference]: #definition-tooltip "The initial, fixed start for our problem. Typically this refers to |0>, but we can configure it to a variety of states that suitably encode our problem."

## Default state

A *reference state* refers to the initial fixed start for our problem. To prepare a reference state, we need to apply the appropriate, non-parametrized unitary $U_R$ at the start of our quantum circuit, such that $|\rho\rangle = U_R |0\rangle$. If you have an educated guess or datapoint from an existing optimal solution, the variational algorithm will likely converge faster if you use that as a starting point.

The simplest possible reference state is the default state, where we use the starting state of an $n$-qubit quantum circuit: $|0\rangle^{\otimes n}$. For the default state, our unitary operator $U_R \equiv I$. Due to its simplicity, the default state is a valid reference state used in many scenarios.

## Classical reference state

Suppose you have a three-qubit system and you want to start in the state $|001\rangle$ instead of the default state $|000\rangle$. This is an example of a purely classical reference state, and to construct it, you simply need to apply an [X gate](/docs/api/qiskit/qiskit.circuit.library.XGate) to qubit $0$ (following Qiskit's qubit ordering), as $|001\rangle = X_0 |000\rangle$.

In this case, our unitary operator is $U_R \equiv X_0$, which leads to the reference state $|\rho\rangle \equiv |001\rangle$.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3)
qc.x(0)

qc.draw("mpl")

<Image src="/learning/images/courses/variational-algorithm-design/reference-states/extracted-outputs/cfd71225-1f9f-4a34-b825-ae19813956a8-0.avif" alt="Output of the previous code cell" />

## Quantum reference state

Suppose you aim to start with a more complex state that involves superposition and/or entanglement, such as $\frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$.

To obtain this state from $|000\rangle$, one approach is to use a [Hadamard gate](/docs/api/qiskit/qiskit.circuit.library.HGate) on qubit $0$ ($H_0$), a [CNOT (CX)](/docs/api/qiskit/qiskit.circuit.library.CXGate) gate with qubit $0$ as the control qubit and qubit $1$ as the target qubit ($CNOT_{01}$), and finally an $X$ gate applied to qubit $2$ ($X_2$).

In this scenario, our unitary operator is $U_{R} \equiv X_2CNOT_{01}H_0|000\rangle$, and our reference state is $|\rho\rangle \equiv \frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$.

In [2]:
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.x(2)

qc.draw("mpl")

<Image src="/learning/images/courses/variational-algorithm-design/reference-states/extracted-outputs/77297158-60d7-469a-8a82-27d38a52d795-0.avif" alt="Output of the previous code cell" />

## Constructing Reference States using template circuits

We can also use various template circuits, such as [`TwoLocal`](/docs/api/qiskit/qiskit.circuit.library.TwoLocal) which allows for expressing multiple tunable parameters and entanglements with ease. We will cover these template circuits in more detail in the next lesson, but we can use them for our reference states *if* we bind the parameters:

In [3]:
from qiskit.circuit.library import TwoLocal
from math import pi

reference_circuit = TwoLocal(2, "rx", "cz", entanglement="linear", reps=1)
theta_list = [pi / 2, pi / 3, pi / 3, pi / 2]

reference_circuit = reference_circuit.assign_parameters(theta_list)

reference_circuit.decompose().draw("mpl")

<Image src="/learning/images/courses/variational-algorithm-design/reference-states/extracted-outputs/8f3948ad-4caf-4ee6-8a72-bb91ed1bfeae-0.avif" alt="Output of the previous code cell" />

## Application-specific reference states

### Quantum machine learning

In the context of a variational quantum classifier (VQC), the training data is encoded into a quantum state with a parameterized circuit known as a *feature map*, where each parameter value represents a data point from the training dataset. The `zz_feature_map` is a type of parameterized circuit that can be utilized to pass our data points ($x$) to this feature map.

In [ ]:
from qiskit.circuit.library import zz_feature_map

data = [0.1, 0.2]

zz_feature_map_reference = zz_feature_map(feature_dimension=2, reps=2)
zz_feature_map_reference = zz_feature_map_reference.assign_parameters(data)
zz_feature_map_reference.decompose().draw("mpl")

<Image src="/learning/images/courses/variational-algorithm-design/reference-states/extracted-outputs/4bfd5a76-0db3-499d-bda1-0cb95c3f23c2-0.avif" alt="Output of the previous code cell" />

## Summary

With this lesson, you learned how to initialize your system using:

- Default reference state
- Classical reference states
- Quantum reference states
- Application-specific reference states

Our high-level variational workload looks as follows:

![A circuit diagram of a unitary operator preparing a reference state.](/learning/images/courses/variational-algorithm-design/reference-states/reference-circuit.svg)

While reference states are fixed, initial starting points, we can use a *variational form* to define an *ansatz* to represent a collection of parametrized states for our variational algorithm to explore.